일반화 성능도 있기 때문에 전체 데이터셋에 대해서 성능 평가를 해봐야 함

In [1]:
import pickle
with open('../models/SVC (C=0.1) norml30_r15.pkl', 'rb') as f:
    classifier = pickle.load(f)
    
from py.utils import load_data
head = 'l30_r15'
directory = '../data/'
x, y, x_words, vocabs = load_data(head, directory)
x = x.tocsr()

x shape = (15105, 2770)
y shape = (15105,)
# features = 2770
# L words = 15105


In [2]:
def get_sample_r(li, topk=5):
    nonzero = x[li,:].nonzero()[1]
    base = min(50, len(nonzero)//2)
    return [vocabs[r] for r in sorted(nonzero, key=lambda x:word_frequency[0,x], reverse=True)[base:base+topk]]

word_frequency = x.sum(axis=0)
word_frequency

matrix([[9769,   64,   14, ...,   31,   34,  316]], dtype=int64)

In [3]:
# sparse matrix type
print(classifier.dual_coef_.shape)

# sum = 0 ?? alpha sum은 1아닌가?
print(classifier.dual_coef_.data.sum())

(1, 5012)
0.0


In [4]:
# Support vectors
classifier.support_vectors_.shape

(5012, 2770)

In [5]:
# Proportion of Support vector 
classifier.n_support_.sum() / x.shape[0]

0.33181065872227739

In [6]:
# Vector row id
classifier.support_

array([    0,     9,    14, ..., 15082, 15085, 15087], dtype=int32)

In [7]:
base = 0
topk = 40
for idx, l in sorted(enumerate(classifier.support_), key=lambda x:abs(classifier.dual_coef_[0,x[0]]), reverse=True)[base :base + topk]:
    print('%s%s - %s (%.3f)' % (x_words[l], '' if y[l] == 1 else '(다)', get_sample_r(l), classifier.dual_coef_[0,idx]))

행해(다) - ['오던', '진다는', '졌을', '지던', '졌다는'] (-0.100)
올라서(다) - ['다가', '기도', '니', '자', '려고'] (-0.100)
거들(다) - ['러', '었고', '었다는', '었다고', '었지만'] (-0.100)
하(다) - ['기가', '든', '기만', '든지', '려'] (-0.100)
심심찮(다) - ['게'] (-0.100)
터져(다) - ['버리는', '나가는', '날', '나갈', '나오는'] (-0.100)
거쳤(다) - ['는데', '으며', '는지', '으나', '음을'] (-0.100)
덩달(다) - ['아서'] (-0.100)
빌리(다) - ['러', '려는', '곤', '기가', '든지'] (-0.100)
계셨(다) - ['는지', '더라도', '으니', '기에', '으니까'] (-0.100)
우스꽝스러(다) - ['움을', '움이'] (-0.100)
모아(다) - ['져야', '지게', '두고', '지면', '잡은'] (-0.100)
지켜보(다) - ['곤', '기가', '기만', '려', '았을'] (-0.100)
끼어들(다) - ['기를', '었기', '었지만', '려', '긴'] (-0.100)
찌들(다) - ['었던', '구', '었고', '어진', '려'] (-0.100)
굽히(다) - ['기도', '자', '거나', '지는', '려는'] (-0.100)
빼내(다) - ['고는', '러', '려는', '듯', '기는'] (-0.100)
아름다(다) - ['우며', '움의', '워도', '움에', '우면서도'] (-0.100)
앞섰(다) - ['다고', '다', '지만', '으나', '음을'] (-0.100)
마셔(다) - ['버리고', '서가', '대는', '서인지', '줘야'] (-0.100)
망설이(다) - ['자', '거나', '지도', '는지', '더니'] (-0.100)
버둥거리(다) - ['면서', '다', '는데', '다가', '기도'] (-0.100)
죽치(다) 

In [8]:
from collections import Counter
alpha_count = Counter(classifier.dual_coef_.data)
print('number of alphas = %d' % len(alpha_count))
print('number of support vectors = %d\n' % len(classifier.dual_coef_.data))

for alpha, count in sorted(alpha_count.items(), key=lambda x:(x[1], abs(x[0])), reverse=True)[:50]:
    print('alpha= %.3f, count=%d (%.3f)' % (alpha, count, count/classifier.n_support_.sum()))

number of alphas = 2
number of support vectors = 5012

alpha= -0.100, count=2506 (0.500)
alpha= 0.100, count=2506 (0.500)


In [9]:
classifier.n_support_

array([2506, 2506], dtype=int32)

Support Vector Machine은 약 30%의 매우 높은 support vector를 모델에 저장하고 있으며, 각각 support vector의 가중치 역시 다르지 않다. negative는 모두 -0.1, positive는 모두 0.1 weight를 가지고 있다. k-NN classifier와 다르지 않을 정도의 support vector ratio이며, 질 좋은 support vector를 선택하지도 (equal alpha) 않는다. 

## Regularization을 약하게 하는 경우

In [10]:
with open('../models/Support Vector Machine (rbf, C=10.0) norml30_r15.pkl', 'rb') as f:
    classifier = pickle.load(f)
    
from py.utils import load_data
head = 'l30_r15'
directory = '../data/'
x, y, x_words, vocabs = load_data(head, directory)
x = x.tocsr()

x shape = (15105, 2770)
y shape = (15105,)
# features = 2770
# L words = 15105


In [11]:
# Proportion of Support vector 
classifier.n_support_.sum() / x.shape[0]

0.20231711353856338

In [12]:
from collections import Counter
alpha_count = Counter(classifier.dual_coef_.data)
print('number of alphas = %d' % len(alpha_count))
print('number of support vectors = %d\n' % len(classifier.dual_coef_.data))

for alpha, count in sorted(alpha_count.items(), key=lambda x:(x[1], abs(x[0])), reverse=True)[:50]:
    print('alpha= %.3f, count=%d (%.3f)' % (alpha, count, count/classifier.n_support_.sum()))

number of alphas = 21
number of support vectors = 3056

alpha= -10.000, count=1521 (0.498)
alpha= 10.000, count=1516 (0.496)
alpha= -9.387, count=1 (0.000)
alpha= -9.067, count=1 (0.000)
alpha= 8.963, count=1 (0.000)
alpha= 8.795, count=1 (0.000)
alpha= 8.257, count=1 (0.000)
alpha= 7.931, count=1 (0.000)
alpha= 6.875, count=1 (0.000)
alpha= -6.653, count=1 (0.000)
alpha= 6.621, count=1 (0.000)
alpha= 6.444, count=1 (0.000)
alpha= 6.254, count=1 (0.000)
alpha= 5.404, count=1 (0.000)
alpha= 3.746, count=1 (0.000)
alpha= 3.379, count=1 (0.000)
alpha= 3.113, count=1 (0.000)
alpha= -2.526, count=1 (0.000)
alpha= 2.069, count=1 (0.000)
alpha= -1.434, count=1 (0.000)
alpha= 1.218, count=1 (0.000)


Regularization 을 약하게 걸어도 대부분은 -10, 10의 weight를 가지며, support vector ratio가 20%로 줄어들었지만 여전히 많은 숫자이다. instance learning과 다르지 않다